<a href="https://colab.research.google.com/github/shakasom/NearestNeighbour-Analysis/blob/master/NNA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [19]:
# From:  https://github.com/shakasom/NearestNeighbour-Analysis/blob/master/NNA.ipynb
!wget https://www.dropbox.com/s/xcxw2hl2zt3fwzg/bike_data.zip

--2020-06-25 22:39:27--  https://www.dropbox.com/s/xcxw2hl2zt3fwzg/bike_data.zip
Resolving www.dropbox.com (www.dropbox.com)... 162.125.3.1, 2620:100:6018:1::a27d:301
Connecting to www.dropbox.com (www.dropbox.com)|162.125.3.1|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: /s/raw/xcxw2hl2zt3fwzg/bike_data.zip [following]
--2020-06-25 22:39:27--  https://www.dropbox.com/s/raw/xcxw2hl2zt3fwzg/bike_data.zip
Reusing existing connection to www.dropbox.com:443.
HTTP request sent, awaiting response... 302 Found
Location: https://ucfdf80fe0c00349c907d3387825.dl.dropboxusercontent.com/cd/0/inline/A6X2G7fnkZGnXbKQ4xPUoRnPbpgc6P1ggV6lDBDscjYUtdlWiHj8_4Q2CCjzvbNlmDCo-uE0vMNiA4CZvpOnPqPvUEMjJCc2yKz8RnjWR4HFiym0J2b6WMaos7Gid7AdVJM/file# [following]
--2020-06-25 22:39:28--  https://ucfdf80fe0c00349c907d3387825.dl.dropboxusercontent.com/cd/0/inline/A6X2G7fnkZGnXbKQ4xPUoRnPbpgc6P1ggV6lDBDscjYUtdlWiHj8_4Q2CCjzvbNlmDCo-uE0vMNiA4CZvpOnPqPvUEMjJCc2yKz8RnjWR4HFiym

In [3]:
!unzip bike_data.zip

Archive:  bike_data.zip
  inflating: points.csv              
  inflating: stations.csv            


In [4]:
import pandas as pd
import geopandas as gpd 
import matplotlib.pyplot as plt
import folium 
from shapely.ops import nearest_points
from shapely.geometry import LineString

In [5]:
stations = pd.read_csv("stations.csv")
stations.head()

,id,name,dpcapacity,xcoord,ycoord
0,5,State St & Harrison St,19,-87.627739,41.873958
1,13,Wilton Ave & Diversey Pkwy,19,-87.652681,41.932500
2,14,Morgan St & 18th St,15,-87.651073,41.858086
3,15,Racine Ave & 19th St,15,-87.656471,41.856453
4,16,Wood St & North Ave,15,-87.672516,41.910329


In [6]:
points = pd.read_csv("points.csv")
points.head()

,id,xcoord,ycoord
0,1,-87.675992,41.969792
1,2,-87.676702,41.956395
2,3,-87.601501,41.805379
3,4,-87.616656,41.858263
4,5,-87.706869,41.963010


In [7]:
def create_gdf(df, x="xcoord", y="ycoord"):
    return gpd.GeoDataFrame(df, geometry=gpd.points_from_xy(df[x], df[y]), crs={"init":"epsg:4326"})

In [8]:
stations_gdf = create_gdf(stations)
points_gdf = create_gdf(points)

/home/ubuntu/ubermedia/env/lib/python3.8/site-packages/pyproj/crs/crs.py:53: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6
  return _prepare_from_string(" ".join(pjargs))


In [9]:
m = folium.Map([41.805379, -87.601501],
               zoom_start=12,
               tiles="CartoDb dark_matter")
locs_stations = zip(stations_gdf.ycoord, stations_gdf.xcoord)
locs_points = zip(points_gdf.ycoord, points_gdf.xcoord)
for location in locs_stations:
    folium.CircleMarker(location=location, color="red", radius=4).add_to(m)
for location in locs_points:
    folium.CircleMarker(location=location, color="white", radius=2).add_to(m)
m.save("map1.html")
m

In [10]:
def calculate_nearest(row, destination, val, col="geometry"):
    dest_unary = destination["geometry"].unary_union
    nearest_geom = nearest_points(row[col], dest_unary)
    match_geom = destination.loc[destination.geometry == nearest_geom[1]]
    match_value = match_geom[val].to_numpy()[0]
    return match_value

In [11]:
points_gdf["nearest_geom"] = points_gdf.apply(calculate_nearest, destination=stations_gdf, val="geometry", axis=1)

In [12]:
points_gdf["nearest_station"] = points_gdf.apply(calculate_nearest, destination=stations_gdf, val="name", axis=1)

In [13]:
points_gdf.head()

,id,xcoord,ycoord,geometry,nearest_geom,nearest_station
0,1,-87.675992,41.969792,POINT (-87.67599 41.96979),POINT (-87.67423700000001 41.96909),Ravenswood Ave & Lawrence Ave
1,2,-87.676702,41.956395,POINT (-87.67670 41.95640),POINT (-87.679259 41.955927),Lincoln Ave & Belle Plaine Ave
2,3,-87.601501,41.805379,POINT (-87.60150 41.80538),POINT (-87.59938299999999 41.809835),Greenwood Ave & 47th St
3,4,-87.616656,41.858263,POINT (-87.61666 41.85826),POINT (-87.619407 41.857611),Calumet Ave & 18th St
4,5,-87.706869,41.963010,POINT (-87.70687 41.96301),POINT (-87.68848699999999 41.966555),Western Ave & Leland Ave


In [14]:
points_gdf['line'] = points_gdf.apply(lambda row: LineString([row['geometry'], row['nearest_geom']]), axis=1)
points_gdf.head()

,id,xcoord,ycoord,geometry,nearest_geom,nearest_station,line
0,1,-87.675992,41.969792,POINT (-87.67599 41.96979),POINT (-87.67423700000001 41.96909),Ravenswood Ave & Lawrence Ave,LINESTRING (-87.67599211881929 41.969792417635...
1,2,-87.676702,41.956395,POINT (-87.67670 41.95640),POINT (-87.679259 41.955927),Lincoln Ave & Belle Plaine Ave,LINESTRING (-87.6767023973826 41.9563952204838...
2,3,-87.601501,41.805379,POINT (-87.60150 41.80538),POINT (-87.59938299999999 41.809835),Greenwood Ave & 47th St,"LINESTRING (-87.601501134953 41.8053785205414,..."
3,4,-87.616656,41.858263,POINT (-87.61666 41.85826),POINT (-87.619407 41.857611),Calumet Ave & 18th St,LINESTRING (-87.61665568206151 41.858262512299...
4,5,-87.706869,41.963010,POINT (-87.70687 41.96301),POINT (-87.68848699999999 41.966555),Western Ave & Leland Ave,LINESTRING (-87.7068694739994 41.9630104417897...


In [15]:
line_gdf = points_gdf[["id", "nearest_station", "line"]].set_geometry('line')

In [16]:
line_gdf.crs = crs={"init":"epsg:4326"}

/home/ubuntu/ubermedia/env/lib/python3.8/site-packages/pyproj/crs/crs.py:53: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6
  return _prepare_from_string(" ".join(pjargs))


In [17]:
points_gdf.drop(["nearest_geom", "line"], axis=1, inplace=True)
points_gdf.head()

,id,xcoord,ycoord,geometry,nearest_station
0,1,-87.675992,41.969792,POINT (-87.67599 41.96979),Ravenswood Ave & Lawrence Ave
1,2,-87.676702,41.956395,POINT (-87.67670 41.95640),Lincoln Ave & Belle Plaine Ave
2,3,-87.601501,41.805379,POINT (-87.60150 41.80538),Greenwood Ave & 47th St
3,4,-87.616656,41.858263,POINT (-87.61666 41.85826),Calumet Ave & 18th St
4,5,-87.706869,41.963010,POINT (-87.70687 41.96301),Western Ave & Leland Ave


In [18]:
m = folium.Map([41.805379, -87.601501],
               zoom_start = 12, 
               tiles="CartoDb dark_matter")
locs_stations = zip(stations_gdf.ycoord, stations_gdf.xcoord)
locs_points = zip(points_gdf.ycoord, points_gdf.xcoord)
for location in locs_stations:
    folium.CircleMarker(location=location, color="red", radius=8).add_to(m)
for location in locs_points:
    folium.CircleMarker(location=location, color="white", radius=4).add_to(m)
folium.GeoJson(line_gdf).add_to(m)
m.save("map2.html")
m

# End